In [111]:
import numpy as np
import gzip, pickle as cPickle
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
# For Tensorflow & Keras
import tensorflow as tf
import keras.layers as lyr
from keras.models import Model
#For Padding
from keras.preprocessing.sequence import pad_sequences
#For XGBoost
import xgboost as xgb
#For Word2Vec
from gensim.models import Word2Vec
#from gensim.models import KeyedVectors

In [142]:
# DEFINE GLOBAL VALUES
STOPS = prep_stops()
Train_Path = './input/df10.csv' # JUST TO TEST FASTER , normally train.csv
Test_Path = './input/test1000.csv'
#Train_Path = './input/train.csv'
#Test_Path = './input/test.csv'
num_voc = 5000
EMBEDDING_DIM=300
lstm_units = 256 # Lower if network takes too long
max_seq_length = 10
num_dense = 100
rate_drop_lstm = 0.3
rate_drop_dense = 0.25



# LOAD DATA
X_train, X_traintest, Y_train, Y_traintest = splitData()
X_test, test_idxs = testData()

# 1st Stage, Question to Int Array
# Build Vocabulary

f1 = BoW()
f1.fit(X_train.append(X_traintest), num_voc)
X_all = X_train.append(X_traintest)
mycv = f1.getVectorizer()
#words to tokens
X_test_q1 = create_padded_seqs(X_test.question1,mycv, max_seq_length)
X_test_q2 = create_padded_seqs(X_test.question2, mycv, max_seq_length)
X_train_q1 = create_padded_seqs(X_train.question1, mycv, max_seq_length)
X_train_q2 = create_padded_seqs(X_train.question2, mycv, max_seq_length)
X_traintest_q1 = create_padded_seqs(X_traintest.question1, mycv, max_seq_length)
X_traintest_q2 = create_padded_seqs(X_traintest.question2, mycv, max_seq_length)

Creating stopwords
SPLIT DATA
LOAD TRAIN DATA
LOAD TEST DATA
FIT CORPUS ON COUNT VECTORIZER


In [114]:
# Create Embedding Matrix: Voc Entity --> Vector
MODEL_FILE = './input/300features_40minwords_10context.bin'
#model = Word2Vec.load(EMBEDDING_FILE)
model = Word2Vec.load(MODEL_FILE)
vocab = model.wv.vocab
#word2vec = KeyedVectors.load_word2vec_format(MODEL_FILE, \
#        binary=True)
print "Loaded model"
print('Preparing embedding matrix')

nb_words = num_voc

embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in mycv.vocabulary_.iteritems():
    if word in vocab:
        embedding_matrix[i] = model[word]
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Loaded model
Preparing embedding matrix
Null word embeddings: 54


In [115]:
embedding_matrix.shape

(5000, 300)

In [23]:
# LOAD VECTORS AS INPUT ---> DOESN'T WORK WITH EMBEDDING LAYER
# question1 = embedding[0:len(embedding)/2]
# question2 = embedding[len(embedding)/2:]
# idx = range(0,len(question1))
# # 3rd Stage LOAD THE MEAN-VECTORS
# EMBEDDING_FILE = './input/arr0_1000.npy'
# embedding = load_zipped_pickle(EMBEDDING_FILE)
# LOAD DATA for Labels
# Train_Path = './input/df10.csv' # JUST TO TEST FASTER , normally train.csv
# Test_Path = './input/test1000.csv'
# train = load_train()

# X_train_vec1, X_traintest_vec1, X_train_vec2, X_traintest_vec2, Y_train, Y_traintest = train_test_split(question1, question2,\
#                                                         train["is_duplicate"],\
#                                                         train_size=0.7, random_state=42)

LOAD TRAIN DATA


In [143]:
# Feed NN with Padded Sequences
embedding_layer = lyr.Embedding(num_voc,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=max_seq_length,
        trainable=False)
lstm_layer = lyr.LSTM(lstm_units, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)

input1_tensor = lyr.Input(X_train_q1.shape[1:])
input2_tensor = lyr.Input(X_train_q2.shape[1:])
e1 = embedding_layer(input1_tensor)
e2 = embedding_layer(input2_tensor)
lstm1 = lstm_layer(e1)
lstm2 = lstm_layer(e2)
#e1 = lyr.Embedding(output_dim=10, input_dim=300, input_length=1)(input1_tensor) # Input Dimension = max_features of CountVec
#e2 = lyr.Embedding(output_dim=10, input_dim=300, input_length=1)(input2_tensor)
#lstm1 = lyr.LSTM(lstm_units, activation='tanh')(e1)
#lstm2 = lyr.LSTM(lstm_units, activation='tanh')(e2)
# Feed input1 and input2 seperately to LSTM:
merge_layer = lyr.concatenate([lstm1, lstm2])  ### MERGE LAYER
merge_layer = lyr.Dropout(rate_drop_dense)(merge_layer)
merge_layer = lyr.BatchNormalization()(merge_layer)
dense_layer = lyr.Dense(num_dense, activation='relu')(merge_layer) # DENSE LAYER
dense_layer = lyr.Dropout(rate_drop_dense)(dense_layer)
dense_layer = lyr.BatchNormalization()(dense_layer)
output_layer = lyr.Dense(1, activation='sigmoid')(dense_layer)
model = Model([input1_tensor, input2_tensor], output_layer)
model.compile(loss='binary_crossentropy', optimizer='adam')
model.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_17 (InputLayer)            (None, 10)            0                                            
____________________________________________________________________________________________________
input_18 (InputLayer)            (None, 10)            0                                            
____________________________________________________________________________________________________
embedding_11 (Embedding)         (None, 10, 300)       1500000     input_17[0][0]                   
                                                                   input_18[0][0]                   
____________________________________________________________________________________________________
lstm_9 (LSTM)                    (None, 256)           570368      embedding_11[0][0]      

In [122]:
# embedding_layer = Embedding(nb_words,
#         EMBEDDING_DIM,
#         weights=[embedding_matrix],
#         input_length=MAX_SEQUENCE_LENGTH,
#         trainable=False)
# lstm_layer = LSTM(num_lstm, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)

# sequence_1_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
# embedded_sequences_1 = embedding_layer(sequence_1_input)
# x1 = lstm_layer(embedded_sequences_1)

# sequence_2_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
# embedded_sequences_2 = embedding_layer(sequence_2_input)
# y1 = lstm_layer(embedded_sequences_2)

# merged = concatenate([x1, y1])
# merged = Dropout(rate_drop_dense)(merged)
# merged = BatchNormalization()(merged)

# merged = Dense(num_dense, activation=act)(merged)
# merged = Dropout(rate_drop_dense)(merged)
# merged = BatchNormalization()(merged)

# preds = Dense(1, activation='sigmoid')(merged)

In [144]:
# FIT --> Train NN weights
model.fit([X_train_q1, X_train_q2], Y_train, 
          validation_data=([X_traintest_q1, X_traintest_q2], Y_traintest), 
          batch_size=128, epochs=14, verbose=2)

# (1) Take Features from Merge Layer and(2) feed to Classifier (XGBoost)
#(1)
mergeNN = Model([input1_tensor, input2_tensor], merge_layer)
mergeNN.compile(loss='mse', optimizer='adam')

F_train = mergeNN.predict([X_train_q1, X_train_q1], batch_size=128)
F_traintest = mergeNN.predict([X_traintest_q1, X_traintest_q2], batch_size=128)
F_test = mergeNN.predict([X_test_q1, X_test_q2], batch_size=128)

#(2)
dTrain = xgb.DMatrix(F_train, label=Y_train)
dTraintest = xgb.DMatrix(F_traintest, label=Y_traintest)
dTest = xgb.DMatrix(F_test)

xgb_params = {
    'objective': 'binary:logistic',
    'booster': 'gbtree',
    'eval_metric': 'logloss',
    'eta': 0.1, 
    'max_depth': 9,
    'subsample': 0.9,
    'colsample_bytree': 1 / F_train.shape[1]**0.5,
    'min_child_weight': 5,
    'silent': 1
}
bst = xgb.train(xgb_params, dTrain, 1000,  [(dTrain,'train'), (dTraintest,'val')], 
                verbose_eval=10, early_stopping_rounds=10)

Train on 7000 samples, validate on 3000 samples
Epoch 1/14
24s - loss: 0.7648 - val_loss: 0.6629
Epoch 2/14
27s - loss: 0.6661 - val_loss: 0.6623
Epoch 3/14
29s - loss: 0.6380 - val_loss: 0.6589
Epoch 4/14
31s - loss: 0.6156 - val_loss: 0.6477
Epoch 5/14
29s - loss: 0.5936 - val_loss: 0.6375
Epoch 6/14
32s - loss: 0.5743 - val_loss: 0.6273
Epoch 7/14
35s - loss: 0.5726 - val_loss: 0.6222
Epoch 8/14
32s - loss: 0.5658 - val_loss: 0.6130
Epoch 9/14
37s - loss: 0.5544 - val_loss: 0.6051
Epoch 10/14
38s - loss: 0.5449 - val_loss: 0.5957
Epoch 11/14
34s - loss: 0.5383 - val_loss: 0.5913
Epoch 12/14
31s - loss: 0.5318 - val_loss: 0.5892
Epoch 13/14
30s - loss: 0.5267 - val_loss: 0.5990
Epoch 14/14
33s - loss: 0.5164 - val_loss: 0.6110
[0]	train-logloss:0.66668	val-logloss:0.678459
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 10 rounds.
[10]	train-logloss:0.492024	val-logloss:0.607801
[20]	train-logloss

In [124]:
# PREDICT Submission File
dTest = xgb.DMatrix(F_test)
df_sub = pd.DataFrame({
        'test_id': test_idxs,
        'is_duplicate': bst.predict(dTest, ntree_limit=bst.best_ntree_limit)
    }).set_index('test_id')


print("Create Submission File")
df_sub.to_csv('newsub.csv')

Create Submission File


# Helper functions

In [109]:
def prep_stops():
	print "Creating stopwords"
	from nltk.corpus import stopwords
	orig_stops = set(stopwords.words("english"))
	not_stops = set(["what", "how", "who"]) # add more if you like
	stops = orig_stops - not_stops		# Set difference
	return stops

def create_padded_seqs(texts, mycv,max_seq_length):
    seqs = texts.apply(text2ints, cv=mycv)
    return pad_sequences(seqs, maxlen=max_seq_length)

def text2ints(text, cv):
    other_index = len(cv.vocabulary_)
    intseq = []
    mysplit = text.split(" ")
    filter(lambda x: x!="", mysplit)
    for word in mysplit:
        intseq.append(cv.vocabulary_[word]) if word in cv.vocabulary_ else other_index
    return intseq

def load_train():
    print("LOAD TRAIN DATA")
    train = pd.read_csv(Train_Path)
    train['question1'].fillna('', inplace=True)
    train['question2'].fillna('', inplace=True)
    train = cleanquestions(train)
    return train
def load_test():
    print("LOAD TEST DATA")
    test = pd.read_csv(Test_Path)
    test['question1'].fillna('', inplace=True)
    test['question2'].fillna('', inplace=True)
    test = cleanquestions(test)
    return test

def cleanreview(review):
    STOPS = [""]
    review = re.sub("[^a-z]", " ", review.lower()) # letters only
    review = review.split(" ")
    meaningful_words = [w for w in review if not w in STOPS]
    sentence = " ".join(meaningful_words)
    return sentence
    
def cleanquestions(df):

    question1 = df.question1.apply(lambda x: cleanreview(x))
    question2 = df.question2.apply(lambda x: cleanreview(x))
    df.loc[:, 'question1'] = pd.Series(question1, index=df.index)
    df.loc[:, 'question2'] = pd.Series(question1, index=df.index)
    return df

def splitData(ratio = 0.7): # ratio = train_set, between 0-1.0, default 0.7
    print("SPLIT DATA")
    #time.sleep(1)
    train = load_train()
    X_train, X_test, Y_train, Y_test = train_test_split(train.loc[:,["question1", "question2"]],\
                                                        train["is_duplicate"],\
                                                        train_size=0.7, random_state=42)
    return X_train, X_test, Y_train, Y_test
def trainData():
    print("LOAD TRAIN DATA FOR PREDICT AND SUBMIT")
    train = load_train()
    X = train.loc[:, ["question1", "question2"]]
    Y = train.loc[:]["is_duplicate"]
    return X,Y
def testData():
    test = load_test()
    X_test = test.loc[:,["question1", "question2"]]
    X_ids = test.loc[:]["test_id"]
    return X_test, X_ids

def rprint(str): # Next print overwrites this, i.e use for indicate progress
	sys.stdout.write("PROCESSING: " + str + "            \r")
	sys.stdout.flush()


# serializes data and saves in a (somewhat) compressed format
def save_zipped_pickle(obj, filename, protocol=-1):
    with gzip.open(filename, 'wb') as f:
        cPickle.dump(obj, f, protocol)
        f.close()

# loads compressed data and restores original state
def load_zipped_pickle(filename):	# loads and unpacks
    with gzip.open(filename, 'rb') as f:
        loaded_object = cPickle.load(f)
        f.close()
        return loaded_object   
    
    
class BoW:
    name = "BoW"
    cv = 0

    def fit(self, X, num_vocab):
        corpus = pd.concat([X.question1, X.question2])
        self.cv = CountVectorizer(analyzer='word',min_df = 0,
        max_features=num_vocab, ngram_range=(1,1), preprocessor=None, stop_words=None,
        tokenizer=None)
        print("FIT CORPUS ON COUNT VECTORIZER")
        self.cv.fit(corpus)

    def transform(self, X):
        return self.cv.transform(X)
    def getVectorizer(self):
        return self.cv